In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

# Loading the preprocessed dataset
df = pd.read_json('preprocessed_dataset.json')

# Encode the internal status labels
label_encoder = LabelEncoder()
df['internalStatus_encoded'] = label_encoder.fit_transform(df['internalStatus'])

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['externalStatus'], df['internalStatus_encoded'], test_size=0.2, random_state=42)

# Tokenize the external status descriptions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to the same length
max_seq_length = max(len(seq) for seq in X_train_seq)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_seq_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_seq_length, padding='post')

# Build an LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_seq_length))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')

print(f"Model accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Save the trained model
model.save('status_prediction_model_lstm.pkl')

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("Model training complete")


Epoch 1/10
31/31 [==============================] - 6s 45ms/step - loss: 2.2803 - accuracy: 0.2129 - val_loss: 2.0593 - val_accuracy: 0.2286
Epoch 2/10
31/31 [==============================] - 0s 13ms/step - loss: 1.8353 - accuracy: 0.2538 - val_loss: 1.6147 - val_accuracy: 0.2163
Epoch 3/10
31/31 [==============================] - 0s 13ms/step - loss: 1.5022 - accuracy: 0.3091 - val_loss: 1.3417 - val_accuracy: 0.7429
Epoch 4/10
31/31 [==============================] - 0s 16ms/step - loss: 1.0760 - accuracy: 0.6203 - val_loss: 0.9030 - val_accuracy: 0.7592
Epoch 5/10
31/31 [==============================] - 1s 17ms/step - loss: 0.6838 - accuracy: 0.7943 - val_loss: 0.7010 - val_accuracy: 0.7388
Epoch 6/10
31/31 [==============================] - 1s 18ms/step - loss: 0.5575 - accuracy: 0.8127 - val_loss: 0.6248 - val_accuracy: 0.7714
Epoch 7/10
31/31 [==============================] - 1s 19ms/step - loss: 0.4657 - accuracy: 0.8424 - val_loss: 0.5383 - val_accuracy: 0.8163
Epoch 8/10
31

c:\Users\ap\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


INFO:tensorflow:Assets written to: status_prediction_model_lstm.pkl\assets


INFO:tensorflow:Assets written to: status_prediction_model_lstm.pkl\assets


Model training complete


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

# Loading the preprocessed dataset
df = pd.read_json('preprocessed_dataset.json')

# Encode the internal status labels
label_encoder = LabelEncoder()
df['internalStatus_encoded'] = label_encoder.fit_transform(df['internalStatus'])

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['externalStatus'], df['internalStatus_encoded'], test_size=0.2, random_state=42)

# Tokenize the external status descriptions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to the same length
max_seq_length = max(len(seq) for seq in X_train_seq)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_seq_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_seq_length, padding='post')

# Deep RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_seq_length))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')

print(f"Model accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Save the trained model
model.save('status_prediction_model_deep_rnn.pkl')
with open('tokenizer_deeprnn.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
with open('label_encoder_deeprnn.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)


print("Model training completed")


Epoch 1/10
31/31 [==============================] - 3s 24ms/step - loss: 1.7741 - accuracy: 0.5425 - val_loss: 1.1041 - val_accuracy: 0.7510
Epoch 2/10
31/31 [==============================] - 0s 10ms/step - loss: 0.7428 - accuracy: 0.8229 - val_loss: 0.5517 - val_accuracy: 0.8163
Epoch 3/10
31/31 [==============================] - 0s 9ms/step - loss: 0.3988 - accuracy: 0.9284 - val_loss: 0.3175 - val_accuracy: 0.9592
Epoch 4/10
31/31 [==============================] - 0s 10ms/step - loss: 0.2265 - accuracy: 0.9713 - val_loss: 0.2005 - val_accuracy: 0.9796
Epoch 5/10
31/31 [==============================] - 0s 9ms/step - loss: 0.1568 - accuracy: 0.9795 - val_loss: 0.1452 - val_accuracy: 0.9796
Epoch 6/10
31/31 [==============================] - 0s 9ms/step - loss: 0.1107 - accuracy: 0.9867 - val_loss: 0.1152 - val_accuracy: 0.9796
Epoch 7/10
31/31 [==============================] - 0s 10ms/step - loss: 0.0882 - accuracy: 0.9898 - val_loss: 0.0910 - val_accuracy: 0.9796
Epoch 8/10
31/31

c:\Users\ap\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


INFO:tensorflow:Assets written to: status_prediction_model_deep_rnn.pkl\assets


INFO:tensorflow:Assets written to: status_prediction_model_deep_rnn.pkl\assets


Model training completed
